## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Louie Wang (zw5565)

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target (the dependent variable) series `s_y`

In [2]:
(df_X, s_y) = load_diabetes(return_X_y = True, as_frame = True)

In [ ]:
df_X.shape()

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW2).  You can not use any libraries outside of pandas and numpy. 

In [3]:
def get_linear_regression_model( df_X, s_y ):
    df_X = np.insert(df_X.to_numpy(),0,1, axis=1)
    (beta_hat,residuals,rank,s) = np.linalg.lstsq(df_X,s_y,rcond=-1)
    return beta_hat
get_linear_regression_model(df_X,s_y)

array([ 152.13348416,  -10.01219782, -239.81908937,  519.83978679,
        324.39042769, -792.18416163,  476.74583782,  101.04457032,
        177.06417623,  751.27932109,   67.62538639])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [4]:
def partition_data( df_X, s_y, k ):
    train = {}
    test = {}
    df = pd.concat([df_X, s_y.to_frame()], axis = 1)
    df['partition num'] = np.random.randint(low = 1, high = k+1, size = len(df))
    for i in range(1, k+1):
        train[i] = df_X[df['partition num'] == i]
        test[i] = s_y[df['partition num'] == i]
    return (train, test)


In [5]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )
total = 0
for i in range(1, 6):
    total += len(dict_k_s_y[i])
    print('{}th fold: {} rows'.format(i, len(dict_k_df_X[i])))
print('Total rows: ' + str(total))

1th fold: 100 rows
2th fold: 77 rows
3th fold: 84 rows
4th fold: 94 rows
5th fold: 87 rows
Total rows: 442


In [ ]:
'''
def partition_data( df_X, s_y, k ):
    rand_indx = np.random(low = 1, high = k+1, size = len(df_X))
    dict_k_df_X = {}
    dict_k_s_y = {}
    
    for k in np.unique(rand_indx):
        dict_k_df_X[k] = df_X[rand_indx == k]
        dict_k_s_y[k] = s_y[rand_indx == k]
    return (dict_k_df_X, dict_k_s_y)
    
def check_partition_lengths(d_df_x, d_s_y, df_X):
    tot = 0
    for i in d_df_x.keys():
        tot += len(d_df_x[i])
        print('Fold {:d} length of df is {:d} and length of series is {:d}'.format(i, len(d_df_x[i], len(d_s_y[i]))))
'''

## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [6]:
def get_mae( s_y, s_y_hat):
    n = len(s_y)
    val = [np.abs(s_y[i] - s_y_hat[i])/n for i in range(n)]
    return np.sum(val)

'''
def get_mae(s_y,s_y_hat):
    return np.sum(np.abs(s_y-s_y_hat)/len(s_y))
'''

In [7]:
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

0.333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [12]:
mae = np.array([])
for k in dict_k_df_X.keys():
    X = dict_k_df_X[k]
    y = dict_k_s_y[k]
    
    train_X = df_X[df_X.index.isin(X.index) == False]
    train_y = s_y[s_y.index.isin(y.index) == False]
    beta = get_linear_regression_model(train_X,train_y)
    X = np.insert(X.to_numpy(),0,1, axis=1)
    s_y_hat = np.matmul(X, beta)
        
    mae = np.append(mae, get_mae(y.to_numpy(),s_y_hat))  

In [ ]:
'''
mae = np.array([])
for k in dict_k_df_X.keys():
    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype = float)
    for not_k in dict_k_df_X.keys():
        if not_k != k:
            train_df_X = pd.concat([train_df_X, dict_k_df_X[not_k]])
            train_s_y = train_s_y.append(dict_k_s_y[not_k])
    test_df_X = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
    
    train_df_X = pd.concat([pd.DataFrame({'intercept':np.ones(len(train_df_X))},index=train_df_X.index),train_f_X], axis=1)
    
    this_beta_hat = get_linear_regression_model(train_df_X, train_s_y)
    
    test_df_X = pd.concat([pd.DataFrame({'intercept':np.ones(len(test_df_X))},index=test_df_X.index),test_f_X], axis=1)
    s_y_hat = test_df_X.apply(lambda x: np.matmul(x, this_beta_hat), axis = 1)
    
    mae = np.append(mae, get_mae(y.to_numpy(),s_y_hat))  
'''

In [13]:
print('MAE of 5 folds are:', mae)
print('Min is {},\nMax is {},\nMean is {}'.format(np.min(mae),np.max(mae),np.mean(mae)))

MAE of 5 folds are: [48.98955398 38.24598233 46.51765508 41.89313702 49.87655179]
Min is 38.24598232770997,
Max is 49.87655179336028,
Mean is 45.10457604028806


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your classification target (the dependent variable) series `s_y`

In [14]:
df_X, s_y = load_iris(return_X_y = True, as_frame = True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

In [15]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)
total2 = 0
for i in range(1, 6):
    total2 += len(dict_k_s_y[i])
    print('{}th fold: {} rows'.format(i, len(dict_k_df_X[i])))
print('Total rows: ' + str(total2))

1th fold: 30 rows
2th fold: 30 rows
3th fold: 30 rows
4th fold: 26 rows
5th fold: 34 rows
Total rows: 150


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [16]:
def get_acc( s_1, s_2 ):
    count = 0
    for i in range(len(s_1)):
        if s_1[i] == s_2[i]:
            count += 1
        else:
            count += 0
    return count/len(s_1)

In [17]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [54]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
X_train = {}
y_train = {}
for k in dict_k_df_X.keys():
    mid_acc = np.array([])
    
    for pos_min_impurity in possible_min_impurity_decrease:
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        inner_acc = np.array([])
        clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = pos_min_impurity)
        X = {}
        y = {}
        for j in dict_k_df_X.keys():
            if j == k:
                pass
            
            X[j] = df_X[(df_X.index.isin(dict_k_df_X[k].index) == False) & (df_X.index.isin(dict_k_df_X[j].index) == False)]
            y[j] = s_y[(s_y.index.isin(dict_k_s_y[k].index) == False) & (s_y.index.isin(dict_k_s_y[j].index) == False)]
                
            clf = clf.fit(X[j], y[j])
            s_y_hat = clf.predict(X[j])
            acc = get_acc(s_y_hat, y[j].to_list())
            inner_acc = np.append(inner_acc, acc)
            
        mid_acc = np.append(mid_acc, inner_acc.mean())
        print('\nTesting {} min impurity decrease'.format(pos_min_impurity))
        print('\tAverage accuracy over 4 folds is {}'.format(inner_acc.mean()))
        
    # Use best min impurity decrease to train model
    for i in range(len(mid_acc)):
        if mid_acc[i] == mid_acc.max():
            p = possible_min_impurity_decrease[i]
            break
    print('\nBest min impurity decrease is', p, '\n' + '-'*50)
    clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = p)
    X_train[k] = df_X[df_X.index.isin(dict_k_df_X[k].index) == False]
    y_train[k] = s_y[s_y.index.isin(dict_k_s_y[k].index) == False]
    clf = clf.fit(X_train[k], y_train[k])
    
    # outer accuracy calculation 
    test_pre = clf.predict(X_train[k])
    this_acc = get_acc(test_pre, y_train[k].to_list())
    outer_acc = np.append(outer_acc,this_acc)


Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9749824069492551

Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.9749824069492551

Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.7395686953653307

Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.3688009236351641

Best min impurity decrease is 0.1 
--------------------------------------------------

Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9650681730716368

Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.8156710099510693

Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.7045082192533949

Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.36859420528891085

Best min impurity decrease is 0.1 
--------------------------------------------------

Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9583927098795975

Testing 0.25 min impurity decr

In [ ]:
'''
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

outer_acc = np.array([])
for k in dict_k_df_X.keys():
    train_df_X = pd.DataFrame()
    train_s_y = pd.Series(dtype=float)
    
    ave_pos_min_impurity_acc = np.array([])
    for pos_min_impurity in possible_min_impurity_decrease:
        pos_min_impurity_acc = np.array([])
        print('Testing {:.2f} min impurity decrease'.format(pos_min_impurity))
        
        poss_keys = list(dict_k_df_X.keys())
        poss_keys.remove(k)
        for inner_k in poss_keys:
            inner_train_df_X = pd.DataFrame()
            inner_train_s_y = pd.Series(dtype=float)
            
            for not_inner_k in poss_keys:
                if not_inner_k != inner_k:
                    inner_train_df_X = pd.concat([inner_train_df_X, dict_k_df_X[not_inner_k]])
                    inner_train_s_y = inner_train_s_y.append(dict_k_s_y[not_inner_k])
                    
            inner_test_df_X = dict_k_df_X[inner_k]
            inner_test_s_y = dict_k_s_y[inner_k]
            
            clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease = pos_min_impurity)
            clf = clf.fit(inner_train_df_X, inner_train_s_y)
            
            this_acc = get_acc(inner_test_s_y, clf.predict(inner_test_df_X))
            pos_min_impurity_acc = np.append(pos_min_impurity_acc, this_acc)
        
        avg_pos_min_impurity_acc = np.append(avg_pos_min_impurity_acc, pos_min_impurity_acc.mean())
        print('\tAverage accuracy over {:d} folds is {:.2f}'.format(len(poss_keys), pos_min_impurity_acc.mean()))
    
    indx_best = np.argmin(1 - ave_pos_min_impurity_acc)
    print('\nBest min impurity decrease is {}\n'.format(possible_min_impurity_decrease[indx_best]))

    this_min_impurity_decrease = possible_min_impurity_decrease[indx_best]
    
    for not_k in dict_k_df_X.keys():
        if not_k != k:
            train_df_X = pd.concat([train_df_X, dict_k_df_X[not_k]])
            train_s_y = train_s_y.append(dict_k_s_y[not_k])
            
    test_df_X = dict_k_df_X[k]
    test_s_y = dict_k_s_y[k]
    
    clf = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=this_min_impurity_decrease)
    clf = clf.fit(train_df_X, train_s_y)
    
    this_acc = get_acc(test_s_y, clf.predict(test_df_X))
    outer_acc = np.append(outer_acc,this_acc)
    
'''

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [55]:
print(outer_acc)
print('Min: {}\nMax: {}\nMean: {}'.format(outer_acc.min(), outer_acc.max(), outer_acc.mean()))

[0.975      0.95833333 0.95833333 0.95967742 0.95689655]
Min: 0.9568965517241379
Max: 0.975
Mean: 0.9616481275491287
